In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1918575,2021-12-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1918576,2021-12-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1918577,2021-12-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1918578,2021-12-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
49739,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
49741,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
49743,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
49745,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
49747,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1166047,2021-12-08,Arkansas,Arkansas,5001,3553,62.00,5000,Arkansas,AR,Arkansas,State,3017804
1166049,2021-12-09,Arkansas,Arkansas,5001,3557,62.00,5000,Arkansas,AR,Arkansas,State,3017804
1166051,2021-12-10,Arkansas,Arkansas,5001,3560,62.00,5000,Arkansas,AR,Arkansas,State,3017804
1166053,2021-12-11,Arkansas,Arkansas,5001,3563,62.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1918575,2021-12-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1918576,2021-12-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1918577,2021-12-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1918578,2021-12-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-12-12') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
691,2021-12-12,Snohomish,Washington,74152,812.00,53061,WA,County,822083,9020.01,98.77
1380,2021-12-12,Cook,Illinois,699372,12166.00,17031,IL,County,5150233,13579.42,236.22
2068,2021-12-12,Orange,California,337937,5814.00,6059,CA,County,3175692,10641.37,183.08
2755,2021-12-12,Maricopa,Arizona,828549,13130.00,4013,AZ,County,4485414,18472.07,292.73
3442,2021-12-12,Los Angeles,California,1547746,27330.00,6037,CA,County,10039107,15417.17,272.24
...,...,...,...,...,...,...,...,...,...,...,...
1916999,2021-12-12,Wheeler,Oregon,129,1.00,41069,OR,County,1332,9684.68,75.08
1917425,2021-12-12,King,Texas,26,0.00,48269,TX,County,272,9558.82,0.00
1917820,2021-12-12,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10
1918211,2021-12-12,Loving,Texas,14,0.00,48301,TX,County,169,8284.02,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
691,2021-12-12,Snohomish,Washington,74152,812.00,53061,WA,County,822083,9020.01,98.77,98.77,9020.01
1380,2021-12-12,Cook,Illinois,699372,12166.00,17031,IL,County,5150233,13579.42,236.22,236.22,13579.42
2068,2021-12-12,Orange,California,337937,5814.00,6059,CA,County,3175692,10641.37,183.08,183.08,10641.37
2755,2021-12-12,Maricopa,Arizona,828549,13130.00,4013,AZ,County,4485414,18472.07,292.73,292.73,18472.07
3442,2021-12-12,Los Angeles,California,1547746,27330.00,6037,CA,County,10039107,15417.17,272.24,272.24,15417.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916999,2021-12-12,Wheeler,Oregon,129,1.00,41069,OR,County,1332,9684.68,75.08,75.08,9684.68
1917425,2021-12-12,King,Texas,26,0.00,48269,TX,County,272,9558.82,0.00,0.00,9558.82
1917820,2021-12-12,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10,229.10,7101.95
1918211,2021-12-12,Loving,Texas,14,0.00,48301,TX,County,169,8284.02,0.00,0.00,8284.02


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1219716,2021-12-12,Hale,Alabama,3214,89.00,1065,AL,County,14651,21937.07,607.47,607.47,21937.07,1
1328661,2021-12-12,Clarke,Alabama,4865,86.00,1025,AL,County,23622,20595.21,364.07,364.07,20595.21,2
1219093,2021-12-12,Fayette,Alabama,3343,85.00,1057,AL,County,16302,20506.69,521.41,521.41,20506.69,3
707376,2021-12-12,Franklin,Alabama,6408,108.00,1059,AL,County,31362,20432.37,344.37,344.37,20432.37,4
1029070,2021-12-12,Winston,Alabama,4823,111.00,1133,AL,County,23629,20411.36,469.76,469.76,20411.36,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163558,2021-12-12,Converse,Wyoming,2354,45.00,56009,WY,County,13822,17030.82,325.57,325.57,17030.82,19
1025940,2021-12-12,Johnson,Wyoming,1373,16.00,56019,WY,County,8445,16258.14,189.46,189.46,16258.14,20
1164183,2021-12-12,Sublette,Wyoming,1504,23.00,56035,WY,County,9831,15298.55,233.95,233.95,15298.55,21
1449563,2021-12-12,Lincoln,Wyoming,3003,25.00,56023,WY,County,19830,15143.72,126.07,126.07,15143.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
952581,2021-12-12,Lowndes,Alabama,1785,68.00,1085,AL,County,9726,18352.87,699.16,699.16,18352.87,27,1
403104,2021-12-12,Walker,Alabama,12286,386.00,1127,AL,County,63521,19341.64,607.67,607.67,19341.64,13,2
1219716,2021-12-12,Hale,Alabama,3214,89.00,1065,AL,County,14651,21937.07,607.47,607.47,21937.07,1,3
951327,2021-12-12,Crenshaw,Alabama,2627,79.00,1041,AL,County,13772,19074.93,573.63,573.63,19074.93,15,4
843315,2021-12-12,Dallas,Alabama,5327,211.00,1047,AL,County,37196,14321.43,567.27,567.27,14321.43,62,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025940,2021-12-12,Johnson,Wyoming,1373,16.00,56019,WY,County,8445,16258.14,189.46,189.46,16258.14,20,19
1413042,2021-12-12,Uinta,Wyoming,4091,31.00,56041,WY,County,20226,20226.44,153.27,153.27,20226.44,10,20
1449563,2021-12-12,Lincoln,Wyoming,3003,25.00,56023,WY,County,19830,15143.72,126.07,126.07,15143.72,22,21
948819,2021-12-12,Albany,Wyoming,6888,40.00,56001,WY,County,38880,17716.05,102.88,102.88,17716.05,17,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,9020.01,31,28
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,9020.01,31,28
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,9020.01,31,28
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,9020.01,31,28
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,9020.01,31,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914137,2021-12-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1914138,2021-12-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1914139,2021-12-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1914140,2021-12-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1214656,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21937.07,3,1,1.00,nan
1214657,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21937.07,3,1,0.00,nan
1214658,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21937.07,3,1,0.00,nan
1214659,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21937.07,3,1,0.00,nan
1214660,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,607.47,21937.07,3,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474167,2021-12-08,Crook,Wyoming,1004,21.00,56011,WY,County,7584,13238.40,276.90,276.90,13238.40,11,23,0.00,1.43
1474168,2021-12-09,Crook,Wyoming,1004,21.00,56011,WY,County,7584,13238.40,276.90,276.90,13238.40,11,23,0.00,1.43
1474169,2021-12-10,Crook,Wyoming,1004,21.00,56011,WY,County,7584,13238.40,276.90,276.90,13238.40,11,23,0.00,1.43
1474170,2021-12-11,Crook,Wyoming,1004,21.00,56011,WY,County,7584,13238.40,276.90,276.90,13238.40,11,23,0.00,1.43


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
948767,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18352.87,1,27,1.00,4.00,0.00,nan
948768,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18352.87,1,27,0.00,3.50,0.00,nan
948769,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18352.87,1,27,0.00,3.29,0.00,nan
948770,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18352.87,1,27,0.00,2.43,0.00,nan
948771,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18352.87,1,27,0.00,1.93,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398638,2021-12-08,Teton,Wyoming,5433,14.00,56039,WY,County,23464,23154.62,59.67,59.67,23171.67,23,3,8.00,4.79,0.00,0.00
398639,2021-12-09,Teton,Wyoming,5436,14.00,56039,WY,County,23464,23167.41,59.67,59.67,23171.67,23,3,3.00,5.00,0.00,0.00
398640,2021-12-10,Teton,Wyoming,5437,14.00,56039,WY,County,23464,23171.67,59.67,59.67,23171.67,23,3,1.00,5.07,0.00,0.00
398641,2021-12-11,Teton,Wyoming,5437,14.00,56039,WY,County,23464,23171.67,59.67,59.67,23171.67,23,3,0.00,5.07,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-12-12') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
481688,2021-12-12,Imperial,California,39462,781.00,6025,CA,County,181215,21776.34,430.98,430.98,21776.34,1,3,0.00,95.29,0.00,0.93
238958,2021-12-12,San Bernardino,California,380385,6007.00,6071,CA,County,2180085,17448.17,275.54,275.54,17448.17,2,6,0.00,607.36,0.00,4.50
3442,2021-12-12,Los Angeles,California,1547746,27330.00,6037,CA,County,10039107,15417.17,272.24,272.24,15417.17,3,14,1459.00,1539.71,5.00,14.43
1164737,2021-12-12,Tuolumne,California,7989,148.00,6109,CA,County,54478,14664.64,271.67,271.67,14664.64,4,19,10.00,13.00,2.00,0.50
106425,2021-12-12,Stanislaus,California,92709,1452.00,6099,CA,County,550660,16835.98,263.68,263.68,16835.98,5,8,0.00,100.71,0.00,2.79
73618,2021-12-12,Shasta,California,26472,470.00,6089,CA,County,180080,14700.13,261.00,261.00,14700.13,6,18,0.00,39.64,0.00,2.14
846407,2021-12-12,Inyo,California,2611,46.00,6027,CA,County,18039,14474.19,255.00,255.00,14474.19,7,20,0.00,4.86,0.00,0.07
766541,2021-12-12,Merced,California,46068,685.00,6047,CA,County,277680,16590.32,246.69,246.69,16590.32,8,9,0.00,61.50,0.00,1.07
1027760,2021-12-12,Kings,California,35733,373.00,6031,CA,County,152940,23364.06,243.89,243.89,23364.06,9,2,0.00,49.64,0.00,0.71
84565,2021-12-12,San Joaquin,California,108876,1842.00,6077,CA,County,762148,14285.41,241.69,241.69,14285.41,10,22,0.00,134.00,0.00,0.36


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1807549,2021-12-12,Lassen,California,8029,61.00,6035,CA,County,30573,26261.73,199.52,199.52,26261.73,17,1,0.00,7.57,0.00,0.29
1027760,2021-12-12,Kings,California,35733,373.00,6031,CA,County,152940,23364.06,243.89,243.89,23364.06,9,2,0.00,49.64,0.00,0.71
481688,2021-12-12,Imperial,California,39462,781.00,6025,CA,County,181215,21776.34,430.98,430.98,21776.34,1,3,0.00,95.29,0.00,0.93
132102,2021-12-12,Tulare,California,87521,1103.00,6107,CA,County,466195,18773.47,236.60,236.60,18773.47,11,4,0.00,136.21,0.00,1.50
300789,2021-12-12,Kern,California,159721,1870.00,6029,CA,County,900202,17742.80,207.73,207.73,17742.80,14,5,0.00,228.86,0.00,6.36
238958,2021-12-12,San Bernardino,California,380385,6007.00,6071,CA,County,2180085,17448.17,275.54,275.54,17448.17,2,6,0.00,607.36,0.00,4.50
48446,2021-12-12,Madera,California,26666,315.00,6039,CA,County,157327,16949.41,200.22,200.22,16949.41,16,7,0.00,47.21,0.00,0.29
106425,2021-12-12,Stanislaus,California,92709,1452.00,6099,CA,County,550660,16835.98,263.68,263.68,16835.98,5,8,0.00,100.71,0.00,2.79
766541,2021-12-12,Merced,California,46068,685.00,6047,CA,County,277680,16590.32,246.69,246.69,16590.32,8,9,0.00,61.50,0.00,1.07
1098494,2021-12-12,Glenn,California,4622,51.00,6021,CA,County,28393,16278.66,179.62,179.62,16278.66,21,10,0.00,5.07,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
481688,2021-12-12,Imperial,California,39462,781.00,6025,CA,County,181215,21776.34,430.98,430.98,21776.34,1,3,0.00,95.29,0.00,0.93
238958,2021-12-12,San Bernardino,California,380385,6007.00,6071,CA,County,2180085,17448.17,275.54,275.54,17448.17,2,6,0.00,607.36,0.00,4.50
3442,2021-12-12,Los Angeles,California,1547746,27330.00,6037,CA,County,10039107,15417.17,272.24,272.24,15417.17,3,14,1459.00,1539.71,5.00,14.43
1164737,2021-12-12,Tuolumne,California,7989,148.00,6109,CA,County,54478,14664.64,271.67,271.67,14664.64,4,19,10.00,13.00,2.00,0.50
106425,2021-12-12,Stanislaus,California,92709,1452.00,6099,CA,County,550660,16835.98,263.68,263.68,16835.98,5,8,0.00,100.71,0.00,2.79
73618,2021-12-12,Shasta,California,26472,470.00,6089,CA,County,180080,14700.13,261.00,261.00,14700.13,6,18,0.00,39.64,0.00,2.14
846407,2021-12-12,Inyo,California,2611,46.00,6027,CA,County,18039,14474.19,255.00,255.00,14474.19,7,20,0.00,4.86,0.00,0.07
766541,2021-12-12,Merced,California,46068,685.00,6047,CA,County,277680,16590.32,246.69,246.69,16590.32,8,9,0.00,61.50,0.00,1.07
1027760,2021-12-12,Kings,California,35733,373.00,6031,CA,County,152940,23364.06,243.89,243.89,23364.06,9,2,0.00,49.64,0.00,0.71
84565,2021-12-12,San Joaquin,California,108876,1842.00,6077,CA,County,762148,14285.41,241.69,241.69,14285.41,10,22,0.00,134.00,0.00,0.36


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,38.07
1,03/21/20,Imperial,4,2.21,3,0.00,37.93
2,03/22/20,Imperial,4,2.21,3,0.00,36.86
3,03/23/20,Imperial,4,2.21,3,0.00,31.29
4,03/24/20,Imperial,9,4.97,3,5.00,26.57
...,...,...,...,...,...,...,...
6280,12/08/21,Glenn,4605,16218.79,10,2.00,3.86
6281,12/09/21,Glenn,4621,16275.14,10,16.00,5.00
6282,12/10/21,Glenn,4622,16278.66,10,1.00,5.07
6283,12/11/21,Glenn,4622,16278.66,10,0.00,5.07


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
6389,12/08/21,San Joaquin,1839.00,241.29,10,0.00,0.14
6390,12/09/21,San Joaquin,1841.00,241.55,10,2.00,0.29
6391,12/10/21,San Joaquin,1842.00,241.69,10,1.00,0.36
6392,12/11/21,San Joaquin,1842.00,241.69,10,0.00,0.36


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)